In [1]:
from keras.datasets import boston_housing

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 6us/step


In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

normalizing data is needed for time series that continous value, so after that the result is centered around 0, and has unit standard deviation

In [17]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [6]:
from keras import models
from keras import layers

In [10]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

the model end with single value unit and no activation, this is typical setup for scalar regression (a regression where have to be predict a single continuous value)

mse (mean squared error) loss function is as always widely used for regression problems.

mae for monitoring that have to be update parameter so it will be mae=mean(|pred-true|)

using K-fold cross validation when your dataset is small  typically with K = 4 or 5

In [19]:
# k-fold validation, // is division with int values
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epoch = 100
all_scores = []
for i in range(k):
    print('processing fold #',i)
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)* num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
    train_data[(i+1) * num_val_samples:]
    ])
    partial_train_targets = np.concatenate(
        [train_targets[:i*num_val_samples],
         train_targets[(i+1) * num_val_samples:]
        ],
        axis=0
    )
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
             epochs=num_epoch, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [20]:
all_scores

[2.3305959701538086, 2.146657705307007, 2.6189074516296387, 2.695591688156128]

In [21]:
np.mean(all_scores)

2.4479382038116455